# Exercise 5 Solution - Higher-Order Regression
### Task
Implement a higher order regression model. The class structure is provided and identical to the one provided in exercise 2.1. Add the following four member functions:
- the forward prediction
- the cost function computation
- the gradient computation
- the training algorithm

### Learning goals
- Understand the foundational steps of machine learning by implementing each of the components
- Understand the principle of gradient descent for multiparameter optimization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

generate the data

In [ ]:
np.random.seed(10)  # Generate the data.
xTrain = np.array([-1.0, -0.8, -0.2, 0.5, 0.7])
yTrain = -(xTrain ** 2) + 10.0
yTrain[3] -= 0.2
xTest = np.array([-1.1, -0.6, 0.1, 0.3])
yTest = -(xTest ** 2) + 10.0

model definition

In [ ]:
class HigherOrderRegression:
    def __init__(self, polynomialDegree, regularization=None):
        self.polynomialDegree = polynomialDegree
        self.regularization = regularization
        self.weights = np.zeros(polynomialDegree)
        self.bias = 0

    def forward(self, x):
        y = self.bias
        for i in range(self.polynomialDegree):
            y += self.weights[i] * x ** (i + 1)
        return y

    def costFunction(self, x, y):
        cost = np.mean((self.forward(x) - y) ** 2)
        if self.regularization == None:
            return cost
        else:
            return cost + self.regularization * (np.sum(self.weights ** 2) + self.bias ** 2)

    def gradient(self, x, y):
        gradientWeights = np.zeros(self.polynomialDegree)
        gradientBias = 2 * (self.forward(x) - y)  # holds all values w.r.t. x
        for i in range(self.polynomialDegree):
            gradientWeights[i] = np.mean(gradientBias * x ** (i + 1))
        gradientBias = np.mean(gradientBias)
        if self.regularization != None:
            for i in range(self.polynomialDegree):
                gradientWeights[i] += 2 * self.regularization * self.weights[i]
            # gradientBias += 2 * self.regularization * self.bias # this might seem like a good idea, but is not (see Chapter 2)
        return gradientWeights, gradientBias

    def train(self, epochs, lr, xTrain, yTrain, xTest, yTest):
        for epoch in range(epochs):
            costTrain = self.costFunction(xTrain, yTrain)
            costTest = self.costFunction(xTest, yTest)
            gradientWeights, gradientBias = self.gradient(xTrain, yTrain)
            self.weights -= lr * gradientWeights
            self.bias -= lr * gradientBias
            if epoch % 100 == 0:
                string = (
                    "Epoch: {}/{}\t\tTraining cost = {:.2e}\t\tValidation cost = {:.2e}"
                )
                print(string.format(epoch, epochs, costTrain, costTest))

model training

In [ ]:
polynomialDegree = 9
lr = 1e-1
epochs = 1000
regularization = 1e-2

model = HigherOrderRegression(polynomialDegree)
# model = HigherOrderRegression(polynomialDegree, regularization) # activate the regularization
model.train(epochs, lr, xTrain, yTrain, xTest, yTest)

visualize the prediction

In [ ]:
yTrainPred = model.forward(xTrain)  # not visualized
yTestPred = model.forward(xTest)  # not visualized

x = np.linspace(-1.1, 1, 100)
yPred = model.forward(x)

fig, ax = plt.subplots(figsize=(12, 6))
ax.scatter(xTest, yTest, color="r", label="testing data")
ax.scatter(xTrain, yTrain, color="k", label="training data")
ax.plot(x, yPred, "k", label="prediction")
ax.legend()
plt.show()